In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [3]:
!pip install transformers torch optuna
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


In [4]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
tokenizer.pad_token = tokenizer.eos_token

text_data = """
Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her....
"""
inputs = tokenizer(text_data, return_tensors="pt", truncation=True, padding=True)

In [7]:
epochs_list = [20, 60, 70]
learning_rate = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [8]:
def train_model(model, inputs, epochs):
    model.train()
    for epoch in range(epochs):
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [9]:
for epochs in epochs_list:
    print(f"\nTraining for {epochs} epochs:")
    train_model(model, inputs, epochs)



Training for 20 epochs:
Epoch 1/20, Loss: 2.674227476119995
Epoch 2/20, Loss: 2.0607285499572754
Epoch 3/20, Loss: 1.759233832359314
Epoch 4/20, Loss: 1.3705083131790161
Epoch 5/20, Loss: 1.0295617580413818
Epoch 6/20, Loss: 0.7734828591346741
Epoch 7/20, Loss: 0.5837382674217224
Epoch 8/20, Loss: 0.5839533805847168
Epoch 9/20, Loss: 0.3474285900592804
Epoch 10/20, Loss: 0.2143458127975464
Epoch 11/20, Loss: 0.20819981396198273
Epoch 12/20, Loss: 0.1158403605222702
Epoch 13/20, Loss: 0.13516779243946075
Epoch 14/20, Loss: 0.10559111833572388
Epoch 15/20, Loss: 0.06032758578658104
Epoch 16/20, Loss: 0.032661039382219315
Epoch 17/20, Loss: 0.024486100301146507
Epoch 18/20, Loss: 0.06728069484233856
Epoch 19/20, Loss: 0.0147326635196805
Epoch 20/20, Loss: 0.027622761204838753

Training for 60 epochs:
Epoch 1/60, Loss: 0.03382323309779167
Epoch 2/60, Loss: 0.15507572889328003
Epoch 3/60, Loss: 0.026390401646494865
Epoch 4/60, Loss: 0.01838284358382225
Epoch 5/60, Loss: 0.1581602245569229


In [10]:
def generate_text(seed_text, max_length=50):
    input_ids = tokenizer(seed_text, return_tensors="pt").input_ids
    model.eval()
    with torch.no_grad():
        generated_text_samples = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(generated_text_samples[0], skip_special_tokens=True)

In [11]:
seed_text = "Once upon a time"
generated_text = generate_text(seed_text)
print("\nGenerated Text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the


In [12]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
    epochs = trial.suggest_int('epochs', 10, 80)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(epochs):
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return loss.item()

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("\nBest hyperparameters found:")
print(study.best_params)

[I 2024-11-05 04:08:20,883] A new study created in memory with name: no-name-c9bf3549-253d-4263-b999-1a389a4a6bac
<ipython-input-12-b4770f9023d3>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)
[I 2024-11-05 04:10:20,629] Trial 0 finished with value: 0.0010004137875512242 and parameters: {'learning_rate': 8.002284323174367e-05, 'epochs': 37}. Best is trial 0 with value: 0.0010004137875512242.
[I 2024-11-05 04:12:37,651] Trial 1 finished with value: 0.00011304118379484862 and parameters: {'learning_rate': 4.052510464011167e-05, 'epochs': 47}. Best is trial 1 with value: 0.00011304118379484862.
[I 2024-11-05 04:15:47,069] Trial 2 finished with value: 0.00011885954882018268 and parameters: {'learning_rate': 2.8405276019327423e-05, 'epochs': 64}. Best is trial 1


Best hyperparameters found:
{'learning_rate': 5.82147096097416e-05, 'epochs': 13}
